In [ ]:
import os
import yaml
from pathlib import Path
import re

from bs4 import BeautifulSoup

In [ ]:
# Paramètres
with open("config.yml", "r", encoding="utf-8") as file:
    config = yaml.safe_load(file)

INPUT_DOSSIER_ARTICLES_WIKI = Path(config["camins"]["texte"]["dossier_articles_wiki"])
OUTPUT_FICHIER_WIKI_TEXTE = config["camins"]["texte"]["fichier_wiki_texte"]

In [ ]:
def tractar_texte_brut(texte: str) -> str:
    caracters_latins = r"[A-Za-zÀ-ÿ0-9\s\-\(\)\.\?\!\:\,\;\%\&\'\"]"
    texte = re.sub(r"\[.+\]", "", texte)  # Levar las citaciones
    texte = "".join(re.findall(caracters_latins, texte))  # Levar los caracters que son pas latins
    texte = re.sub(r"\s{2,}", " ", texte)  # Remplaçar espacis consecutius
    texte = texte.lower().replace("\r", " ").replace("\t", " ").replace("\n", " ").replace('"', "").strip()
    return texte


def extraire_texte(path: Path) -> str:
    """
    Extraire e tractar tèxte d'un fichièr HTML.
    """
    with open(path, "r") as fichier:
        sopa = BeautifulSoup(fichier, "html.parser")

    # Extraire tèxte
    texte = " ".join([x.get_text() for x in sopa.find_all("p")])

    # Tractar tèxte
    return tractar_texte_brut(texte=texte)

In [ ]:
# Obténer la lista dels noms d'articles
nom_articles = [x for x in os.listdir(INPUT_DOSSIER_ARTICLES_WIKI) if x and x[0].isalpha()]

with open(OUTPUT_FICHIER_WIKI_TEXTE, "w") as fichier:
    fichier.write(f"Article,Texte\n")
    for nom_article in nom_articles:
        path = INPUT_DOSSIER_ARTICLES_WIKI / Path(nom_article)
        if not path.is_file():
            continue

        # Extraire tèxte de l'article
        texte_article = extraire_texte(path)
        if texte_article:
            fichier.write(f'"{nom_article}","{texte_article}"\n')